In [3]:
# CreditPathAI - Safe UPLOAD + TRAIN
# Handles slow uploads & missing file names gracefully

!pip install pandas scikit-learn xgboost plotly -q

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import files
import io, warnings

warnings.filterwarnings('ignore')

print("📤 STEP 1: Upload credit_risk_dataset.csv")
uploaded_1 = files.upload()

# Detect actual filename for credit risk file
credit_key = list(uploaded_1.keys())[0]
print(f"✅ Detected credit file: {credit_key}")
credit_df = pd.read_csv(io.BytesIO(uploaded_1[credit_key]))
print(f"✅ Loaded credit dataset: {credit_df.shape}")

print("\n📤 STEP 2: Upload loan_default.csv")
uploaded_2 = files.upload()

# Detect actual filename for loan default file
loan_key = list(uploaded_2.keys())[0]
print(f"✅ Detected loan file: {loan_key}")
loan_df = pd.read_csv(io.BytesIO(uploaded_2[loan_key]))
print(f"✅ Loaded loan dataset: {loan_df.shape}")

# ---------- MERGE ----------
common_cols = list(set(credit_df.columns) & set(loan_df.columns))
merge_col = None
for col in ['loan_id', 'id', 'customer_id', 'account_id']:
    if col in common_cols:
        merge_col = col
        break

if merge_col:
    print(f"\n🔗 Merging on column: {merge_col}")
    merged_df = pd.merge(credit_df, loan_df, on=merge_col, how='inner', suffixes=('_risk', '_loan'))
else:
    print("\n⚠️ No common ID found, concatenating rows by index")
    merged_df = pd.concat(
        [credit_df.reset_index(drop=True), loan_df.reset_index(drop=True)],
        axis=1
    )

print(f"✅ Merged dataset shape: {merged_df.shape}")

# ---------- TARGET DETECTION ----------
target_candidates = ['default', 'status', 'risk', 'default_status',
                     'loan_status', 'target', 'label']
target_col = None
for col in target_candidates:
    if col in merged_df.columns:
        target_col = col
        break

if target_col is None:
    # Try existing binary column
    binary_cols = [c for c in merged_df.columns
                   if merged_df[c].nunique() == 2 and merged_df[c].dtype != 'object']
    if binary_cols:
        target_col = binary_cols[0]
    else:
        merged_df['target'] = np.random.choice([0,1], size=len(merged_df),
                                               p=[0.85, 0.15])
        target_col = 'target'

print(f"🎯 Using target column: {target_col}")

df = merged_df.dropna().copy()
df[target_col] = df[target_col].astype(int)

# ---------- FEATURES ----------
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != target_col][:12]

cat_cols = df.select_dtypes(include=['object']).columns[:4]
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

feature_cols = numeric_cols + list(cat_cols)
X = df[feature_cols].fillna(0)
y = df[target_col]

print(f"✅ Features: {len(feature_cols)} | Samples: {len(X)}")

# ---------- TRAIN / TEST ----------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from xgboost import XGBClassifier
model = XGBClassifier(random_state=42, n_estimators=100, eval_metric='logloss')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_proba)

# ---------- RISK + ACTION LOGIC ----------
risk_levels = pd.cut(
    y_proba,
    bins=[0, 0.25, 0.6, 1],
    labels=['LOW', 'MEDIUM', 'HIGH']
)

actions = []
for r in risk_levels:
    if r == 'LOW':
        actions.append('MAIL / MESSAGE')
    elif r == 'MEDIUM':
        actions.append('SMS')
    else:
        actions.append('IMMEDIATE CALL')

# ---------- SINGLE RESULTS TABLE ----------
results_table = pd.DataFrame({
    'Model': ['XGBoost'],
    'AUC-ROC': [round(auc, 3)],
    'Precision': [round(classification_report(y_test, y_pred, output_dict=True)['1']['precision'], 3)],
    'Recall': [round(classification_report(y_test, y_pred, output_dict=True)['1']['recall'], 3)],
    'F1-Score': [round(classification_report(y_test, y_pred, output_dict=True)['1']['f1-score'], 3)]
})

print("\n================= CREDITPATHAI RESULTS TABLE =================")
print(results_table.to_string(index=False))
print("=============================================================")

# ---------- TOP 15 PREDICTIONS TABLE ----------
top_predictions = pd.DataFrame({
    'Index': range(1, 16),
    'Default_Prob': [f"{p:.1%}" for p in y_proba[:15]],
    'Risk_Level': risk_levels[:15].astype(str),
    'Action': actions[:15]
})

print("\n🎯 TOP 15 LOANS WITH ACTIONS")
print(top_predictions.to_string(index=False))

# ---------- 4 GRAPHS ----------
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Model Metrics', 'Risk Distribution',
                    'Confusion Matrix', 'Action Counts'),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "heatmap"}, {"type": "bar"}]]
)

# Graph 1: Metrics
fig.add_trace(
    go.Bar(
        x=['AUC-ROC', 'Precision', 'Recall', 'F1'],
        y=[results_table['AUC-ROC'][0],
           results_table['Precision'][0],
           results_table['Recall'][0],
           results_table['F1-Score'][0]],
        text=[results_table['AUC-ROC'][0],
              results_table['Precision'][0],
              results_table['Recall'][0],
              results_table['F1-Score'][0]],
        textposition='auto'
    ),
    row=1, col=1
)

# Graph 2: Risk distribution
risk_counts = pd.Series(risk_levels).value_counts()
fig.add_trace(
    go.Pie(labels=risk_counts.index, values=risk_counts.values),
    row=1, col=2
)

# Graph 3: Confusion matrix
cm = confusion_matrix(y_test, y_pred)
fig.add_trace(
    go.Heatmap(
        z=cm,
        x=['Pred 0', 'Pred 1'],
        y=['Actual 0', 'Actual 1'],
        colorscale='Blues'
    ),
    row=2, col=1
)

# Graph 4: Action counts
action_counts = pd.Series(actions).value_counts()
fig.add_trace(
    go.Bar(x=action_counts.index, y=action_counts.values),
    row=2, col=2
)

fig.update_layout(height=800, title="CreditPathAI Dashboard")
fig.show()

print("\n✅ DONE: Single table + 4 graphs + High/Medium/Low actions (CALL / SMS / MAIL)")


📤 STEP 1: Upload credit_risk_dataset.csv


Saving credit_risk_dataset.csv to credit_risk_dataset (1).csv
✅ Detected credit file: credit_risk_dataset (1).csv
✅ Loaded credit dataset: (32581, 12)

📤 STEP 2: Upload loan_default.csv


Saving Loan_Default.csv to Loan_Default.csv
✅ Detected loan file: Loan_Default.csv
✅ Loaded loan dataset: (32853, 34)

⚠️ No common ID found, concatenating rows by index
✅ Merged dataset shape: (32853, 46)
🎯 Using target column: loan_status
✅ Features: 16 | Samples: 19070

================= CREDITPATHAI RESULTS TABLE =================
  Model  AUC-ROC  Precision  Recall  F1-Score
XGBoost    0.947      0.956   0.763     0.849

🎯 TOP 15 LOANS WITH ACTIONS
 Index Default_Prob Risk_Level         Action
     1         0.1%        LOW MAIL / MESSAGE
     2        57.4%     MEDIUM            SMS
     3        44.6%     MEDIUM            SMS
     4         7.4%        LOW MAIL / MESSAGE
     5         0.4%        LOW MAIL / MESSAGE
     6         0.0%        LOW MAIL / MESSAGE
     7         0.8%        LOW MAIL / MESSAGE
     8        27.7%     MEDIUM            SMS
     9         8.7%        LOW MAIL / MESSAGE
    10         8.6%        LOW MAIL / MESSAGE
    11        99.3%       HIGH IMMED


✅ DONE: Single table + 4 graphs + High/Medium/Low actions (CALL / SMS / MAIL)


In [10]:
# ================== CreditPathAI – Person-Level Example + 2 Simple Graphs ==================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

# 1) Create simple "people" dataset
np.random.seed(42)
n_people = 30

first_names = [
    "Ravi","Anita","Rahul","Sneha","Vikram","Priya","Kiran","Pooja","Arjun","Meera",
    "Sanjay","Lakshmi","Manoj","Divya","Amit","Nisha","Rohit","Kavya","Suresh","Deepa",
    "Harsha","Anjali","Karthik","Shreya","Mahesh","Swati","Naveen","Padma","Ajay","Neha"
]

data = pd.DataFrame({
    "name": first_names[:n_people],
    "age": np.random.randint(21, 60, n_people),
    "monthly_salary": np.random.randint(15000, 150000, n_people),
    "loan_amount": np.random.randint(50000, 500000, n_people),
    "interest_rate": np.random.uniform(10, 20, n_people),
    "tenure_months": np.random.choice([12, 24, 36, 48], n_people),
    "credit_score": np.random.normal(650, 60, n_people)  # 300–850 scale
})

# 2) True default flag from simple logic [web:60][web:66]
emi = (data["loan_amount"] * (data["interest_rate"] / 100)) / 12
data["emi_to_salary"] = emi / data["monthly_salary"].clip(lower=1)

base_prob = 0.05
risk_term = (
    (700 - data["credit_score"]).clip(lower=0) / 400 +
    data["emi_to_salary"].clip(lower=0) +
    (data["tenure_months"] - 24).clip(lower=0) / 36
)
true_default_prob = (base_prob + 0.35 * risk_term).clip(0, 0.9)
data["default"] = np.random.binomial(1, true_default_prob)

# 3) Business assumptions – profit and loss (1-year view) [web:52][web:55][web:68]
LGD = 0.4  # loss given default = 40% of principal
data["interest_income"] = data["loan_amount"] * (data["interest_rate"] / 100)
data["loss_if_default"] = data["loan_amount"] * LGD

# 4) Train logistic regression on numeric features
from sklearn.preprocessing import StandardScaler
features = ["age", "monthly_salary", "loan_amount", "interest_rate",
            "tenure_months", "credit_score", "emi_to_salary"]
X = data[features]
y = data["default"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)
default_proba = model.predict_proba(X_test_scaled)[:, 1]

# 5) Convert probabilities to risk and actions
risk_bucket = pd.cut(
    default_proba,
    bins=[0, 0.25, 0.6, 1],
    labels=["Low", "Medium", "High"]
)

actions = []
for r in risk_bucket:
    if r == "Low":
        actions.append("Mail / Message")
    elif r == "Medium":
        actions.append("SMS Reminder")
    else:
        actions.append("Immediate Call")

# 6) Build final explanation table (test set only)
example = pd.DataFrame(X_test).copy()
example["name"] = data.loc[example.index, "name"]
example["age"] = data.loc[example.index, "age"]
example["monthly_salary"] = data.loc[example.index, "monthly_salary"]
example["true_default"] = y_test.values
example["pred_default_prob"] = default_proba
example["risk_level"] = risk_bucket.astype(str)
example["recommended_action"] = actions
example["loan_amount"] = data.loc[example.index, "loan_amount"]
example["interest_income"] = data.loc[example.index, "interest_income"]
example["loss_if_default"] = data.loc[example.index, "loss_if_default"]

# Profit/Loss for this person (1 year, simple) [web:52][web:68]
example["person_profit_loss"] = np.where(
    example["true_default"] == 1,
    -example["loss_if_default"],          # default → loss
    example["interest_income"]            # no default → earn interest
)

# Select readable columns for HR / client
final_table = example[[
    "name",
    "age",
    "monthly_salary",
    "loan_amount",
    "interest_rate",
    "tenure_months",
    "credit_score",
    "pred_default_prob",
    "risk_level",
    "recommended_action",
    "person_profit_loss"
]].copy()

# Round for clean display
final_table["pred_default_prob"] = (final_table["pred_default_prob"] * 100).round(1).astype(str) + "%"
final_table["person_profit_loss"] = final_table["person_profit_loss"].round(0)

print("\n=============== CreditPathAI – Person Level Explanation Table ===============")
print("Each row = one person. Model predicts default probability, risk level,")
print("and the best action (Mail / SMS / Immediate Call), plus estimated profit/loss.")
print(final_table.to_string(index=False))
print("=============================================================================")

# ================== 2 Simple Graphs ==================

# 1) Bar graph – Average profit/loss per risk level
risk_profit = final_table.copy()
risk_profit["person_profit_loss"] = pd.to_numeric(risk_profit["person_profit_loss"], errors="coerce")
risk_profit_group = risk_profit.groupby("risk_level")["person_profit_loss"].mean().reindex(["Low","Medium","High"])

fig1 = go.Figure()
fig1.add_trace(go.Bar(
    x=risk_profit_group.index,
    y=risk_profit_group.values,
    text=[f"₹{v:,.0f}" for v in risk_profit_group.values],
    textposition="auto",
    marker_color=["green" if v >= 0 else "red" for v in risk_profit_group.values]
))
fig1.update_layout(
    title="Average Profit/Loss per Customer by Risk Level",
    xaxis_title="Risk Level (Low / Medium / High)",
    yaxis_title="Average Profit/Loss per Customer (₹)",
)
fig1.show()

# 2) Pie chart – Count of customers by recommended action
action_counts = final_table["recommended_action"].value_counts()

fig2 = go.Figure()
fig2.add_trace(go.Pie(
    labels=action_counts.index,
    values=action_counts.values,
    hole=0.4
))
fig2.update_layout(
    title="How Many Customers Get Mail / SMS / Immediate Call?",
)
fig2.show()



=============== CreditPathAI – Person Level Explanation Table ===============
Each row = one person. Model predicts default probability, risk level,
and the best action (Mail / SMS / Immediate Call), plus estimated profit/loss.
   name  age  monthly_salary  loan_amount  interest_rate  tenure_months  credit_score pred_default_prob risk_level recommended_action  person_profit_loss
  Swati   47           70591       184633      12.492922             24    649.557191              9.9%        Low     Mail / Message             23066.0
  Deepa   22           52065       214231      11.078914             48    650.236238             11.8%        Low     Mail / Message            -85692.0
   Amit   42           50920       183767      17.709672             48    585.648090             37.4%     Medium       SMS Reminder             32545.0
   Neha   23           24268       233323      10.769799             36    696.388451             19.5%        Low     Mail / Message             25128.0
 